# Deep Learning NLP Project: Customer Support Ticket Classification
## Student Practice Notebook

### Project Overview
In this project, you will build an end-to-end Deep Learning solution for automatically classifying customer support tickets into appropriate departments using Natural Language Processing (NLP) techniques. You'll create a hybrid CNN-LSTM neural network model to categorize support tickets into departments (Technical Support, Sales, Billing, Customer Service).

### Learning Objectives:
- Understand text classification for business applications
- Learn hybrid CNN-LSTM architectures
- Master text preprocessing for support tickets
- Build and deploy a ticket routing system
- Evaluate multi-class classification performance

### Business Value:
- Automatic ticket routing to appropriate departments
- Faster customer response times
- Improved support team efficiency
- Better resource allocation

### Instructions:
Complete each code cell by writing the required code based on the comments provided. Each cell contains detailed instructions about what needs to be implemented.

## Step 1: Import Required Libraries

**Task:** Import all necessary libraries for data processing, NLP, and deep learning.

In [2]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import re
import string
import pickle
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Sklearn for preprocessing and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# TensorFlow and Keras for Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

All libraries imported successfully!
TensorFlow version: 2.19.0


## Step 2: Load and Explore Dataset

**Task:** Load the support tickets CSV file and perform exploratory data analysis.

In [3]:
# Load the dataset from 'support_tickets.csv' into a DataFrame called df
df = pd.read_csv("/content/support_tickets.csv")

# Print the shape of the dataset with label "Dataset Shape:"
df.shape

# Print "\nFirst few rows:" and display the first 10 rows using head(10)
df.head(10)


# Print "\nDataset Info:" and display info about the dataset
df.info()


# Print "\nMissing values:" and display count of missing values for each column
df.isnull().sum()


# Print "\nDepartment distribution:" and display value counts of 'department' column
print(df["department"].value_counts())


# Print "\nPriority distribution:" and display value counts of 'priority' column
print(df["priority"].value_counts())


# Print "\nStatus distribution:" and display value counts of 'status' column
print(df["status"].value_counts())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ticket_text  165 non-null    object
 1   priority     165 non-null    object
 2   department   165 non-null    object
 3   status       165 non-null    object
dtypes: object(4)
memory usage: 5.3+ KB
department
Technical Support    68
Customer Service     50
Sales                33
Billing              14
Name: count, dtype: int64
priority
High      67
Low       55
Medium    43
Name: count, dtype: int64
status
Open      134
Closed     31
Name: count, dtype: int64


## Step 3: Data Cleaning and Preprocessing

**Task:** Clean the data by removing duplicates and handling missing values.

In [4]:
# Create a copy of df called df_clean
df_clean = df.copy()

# Print the number of duplicates before removal using duplicated().sum()
print(df_clean.duplicated().sum())

# Remove duplicates based on 'ticket_text' column, keeping the first occurrence
df_clean = df_clean.drop_duplicates(subset='ticket_text', keep='first')

# Print the number of duplicates after removal
print(df_clean.duplicated().sum())

# Drop rows with missing values in 'ticket_text' or 'department' columns
df_clean = df_clean.dropna(subset=["ticket_text", "department"])

# Reset the index of df_clean, dropping the old index
df_clean.reset_index(drop=True, inplace=True)

# Print the final dataset shape
print(df_clean.shape)

# Print "\nDepartment distribution after cleaning:" and display value counts of 'department' column
print("\nDepartment distribution after cleaning:")
print(df_clean['department'].value_counts())



0
0
(165, 4)

Department distribution after cleaning:
department
Technical Support    68
Customer Service     50
Sales                33
Billing              14
Name: count, dtype: int64


## Step 4: Text Preprocessing Functions

**Task:** Create functions to clean and preprocess support ticket text.

In [5]:
# Initialize WordNetLemmatizer and store in variable called lemmatizer


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Keep some important words for support tickets
important_words = {'not', 'no', 'nor', 'very', 'urgent', 'immediately', 'cannot', 'need', 'help', 'issue', 'problem'}
stop_words = stop_words - important_words

def clean_text(text):
    """
    Clean and preprocess text data
    Steps:
    1. Convert to lowercase
    2. Remove URLs
    3. Remove HTML tags
    4. Remove email addresses
    5. Remove special characters and digits
    6. Remove extra whitespace
    """
    # Convert to lowercase
    text = str(text).lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove order numbers and ticket IDs
    text = re.sub(r'#\d+', '', text)

    # Remove special characters and digits but keep apostrophes
    text = re.sub(r"[^a-zA-Z\s']", " ", text)


    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text





# Define a function tokenize_and_lemmatize(text) that:
# 1. Tokenizes the text using word_tokenize()
# 2. Lemmatizes each token using lemmatizer.lemmatize() if token is not in stop_words and length > 2
# 3. Joins the tokens back into a string with spaces
# 4. Returns the processed text

def tokenize_and_lemmatize(text):
      tokens = word_tokenize(text)
      # Lemmatize and remove stopwords + short words
      processed_tokens = [
          lemmatizer.lemmatize(token)
          for token in tokens
          if token not in stop_words and len(token) > 2
      ]
      # Join tokens back into string
      processed_text = " ".join(processed_tokens)
      return processed_text

# Print success message

print("Tokenization and lemmatization function defined successfully!")




# Print success message



Tokenization and lemmatization function defined successfully!


## Step 5: Apply Text Preprocessing

**Task:** Apply the preprocessing functions to all support tickets.

In [6]:
# Print "Cleaning text..."
print("Cleaning Text")

# Create a new column 'cleaned_text' by applying clean_text function to 'ticket_text' column
df_clean['cleaned_text'] = df_clean['ticket_text'].apply(clean_text)

# Print "Tokenizing and lemmatizing..."
print("Tokenizing and lemmatizing...")

# Create a new column 'processed_text' by applying tokenize_and_lemmatize function to 'cleaned_text' column
df_clean['processed_text'] = df_clean['cleaned_text'].apply(tokenize_and_lemmatize)

# Remove rows where 'processed_text' is empty (after stripping whitespace)
df_clean = df_clean[df_clean['processed_text'].str.strip() != '']

# Reset the index, dropping the old index
df_clean = df_clean.reset_index(drop=True)

# Print example showing original, cleaned, and processed text for the first row
# Format: "Original: {original_text}\n\nCleaned: {cleaned_text}\n\nProcessed: {processed_text}"
row = df_clean.iloc[0]
print(f"Original: {row['ticket_text']}\n\nCleaned: {row['cleaned_text']}\n\nProcessed: {row['processed_text']}")



# Print the final dataset shape after preprocessing
print(f"Final dataset shape: {df_clean.shape}")


Cleaning Text
Tokenizing and lemmatizing...
Original: My internet connection keeps dropping every few minutes. This has been happening for the past 3 days. Very frustrating!

Cleaned: my internet connection keeps dropping every few minutes this has been happening for the past days very frustrating

Processed: internet connection keep dropping every minute happening past day very frustrating
Final dataset shape: (165, 6)


## Step 6: Feature Engineering

**Task:** Create additional features from the support ticket text.

In [7]:
# Create a new column 'text_length' with the length of 'processed_text' for each row
df_clean['text_length'] = df_clean['processed_text'].str.len()

# Create a new column 'word_count' with the count of words (split by space) in 'processed_text'
df_clean['word_count'] = df_clean['processed_text'].str.split().str.len()

# Create a new column 'avg_word_length' with the average length of words in 'processed_text'
# Use a lambda function that calculates mean of word lengths, or 0 if no words exist
df_clean['avg_word_length'] = df_clean['processed_text'].apply(
    lambda x: sum(len(w) for w in x.split()) / len(x.split()) if len(x.split()) > 0 else 0
)

# Create a new column 'exclamation_count' counting exclamation marks in 'ticket_text' (urgency indicator)
df_clean['exclamation_count'] = df_clean['ticket_text'].str.count('!')

# Create a new column 'question_count' counting question marks in 'ticket_text'
df_clean['question_count'] = df_clean['ticket_text'].str.count('\?')

# Print "Text statistics:" header
print("Text statistics:")

# Print average text_length with 2 decimal places
print(f"Average text length: {df_clean['text_length'].mean():.2f}")

# Print average word_count with 2 decimal places
print(f"Average word count: {df_clean['word_count'].mean():.2f}")

# Print average avg_word_length with 2 decimal places
print(f"Average word length: {df_clean['avg_word_length'].mean():.2f}")

# Print average exclamation_count with 2 decimal places
print(f"Average exclamation count: {df_clean['exclamation_count'].mean():.2f}")

# Print average question_count with 2 decimal places
print(f"Average question count: {df_clean['question_count'].mean():.2f}")


Text statistics:
Average text length: 73.32
Average word count: 9.56
Average word length: 6.85
Average exclamation count: 0.46
Average question count: 0.19


## Step 7: Prepare Data for Deep Learning Model

**Task:** Prepare features and labels, encode department labels for neural network training.

In [8]:
# Create X as a numpy array from 'processed_text' column values
X = df_clean['processed_text'].values

# Create y as a numpy array from 'department' column values
y = df_clean['department'].values

# Initialize a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform y using the label_encoder, store result in y_encoded
y_encoded = label_encoder.fit_transform(y)

# Print "Label mapping:" header
print("Label mapping:")

# Loop through label_encoder.classes_ with enumerate and print each department label with its index
for index, label in enumerate(label_encoder.classes_):
    print(f"{index}: {label}")


# Convert y_encoded to categorical format using to_categorical(), store in y_categorical
y_categorical = to_categorical(y_encoded)

# Get the number of classes from the shape of y_categorical (second dimension)
num_classes = y_categorical.shape[1]

# Print the number of classes (departments)
print(f"Number of classes: {num_classes}")

# Print the shape of X (features)
print(f"Shape of X: {X.shape}")

# Print the shape of y_categorical (labels)
print(f"Shape of y_categorical: {y_categorical.shape}")


Label mapping:
0: Billing
1: Customer Service
2: Sales
3: Technical Support
Number of classes: 4
Shape of X: (165,)
Shape of y_categorical: (165, 4)


## Step 8: Text Tokenization and Padding

**Task:** Convert text to sequences and pad them to uniform length.

In [9]:
# Define MAX_WORDS constant as 5000 (maximum vocabulary size)
MAX_WORDS = 5000

# Define MAX_SEQUENCE_LENGTH constant as 150 (support tickets are typically longer)
MAX_SEQUENCE_LENGTH = 150

# Initialize a Tokenizer with num_words=MAX_WORDS and oov_token='<OOV>'
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')

# Fit the tokenizer on texts in X
tokenizer.fit_on_texts(X)

# Convert texts to sequences using tokenizer.texts_to_sequences()
X_sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to MAX_SEQUENCE_LENGTH using pad_sequences with padding='post' and truncating='post'
X_padded = pad_sequences(X_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# Print the vocabulary size (length of tokenizer.word_index)
print(f"Vocabulary size: {len(tokenizer.word_index)}")

# Print the shape of X_padded
print(f"Shape of X_padded: {X_padded.shape}")

# Print example: first 10 tokens of the first padded sequence
print(f"First 10 tokens of sequence 0: {X_padded[0][:10]}")


Vocabulary size: 800
Shape of X_padded: (165, 150)
First 10 tokens of sequence 0: [282 150 151 283  90 284 285 152 286   9]


## Step 9: Train-Test Split

**Task:** Split the data into training and testing sets.

In [15]:
# Split X_padded and y_categorical into train and test sets
# Use test_size=0.2, random_state=42, and stratify=y_encoded
# Store results in X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(
    X_padded,
    y_categorical,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)


# Print training set size (number of samples in X_train)
print(f"Training set size: {len(X_train)}")

# Print testing set size (number of samples in X_test)
print(f"Testing set size: {len(X_test)}")

# Print the shape of X_train
print(f"Shape of X_train: {X_train.shape}")

# Print the shape of X_test
print(f"Shape of X_test: {X_test.shape}")


Training set size: 132
Testing set size: 33
Shape of X_train: (132, 150)
Shape of X_test: (33, 150)


## Step 10: Build Hybrid CNN-LSTM Deep Learning Model

**Task:** Create a Sequential model combining CNN and LSTM layers for better ticket classification.

In [16]:
# Define EMBEDDING_DIM constant as 128
EMBEDDING_DIM = 128

# Define LSTM_UNITS constant as 64
LSTM_UNITS = 64

# Define CNN_FILTERS constant as 64
CNN_FILTERS = 64

# Create a Sequential model with the following layers:
# 1. Embedding layer: input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH
# 2. Conv1D layer: filters=CNN_FILTERS, kernel_size=5, activation='relu'
# 3. MaxPooling1D layer: pool_size=2
# 4. Bidirectional LSTM layer: LSTM_UNITS units, return_sequences=True
# 5. Dropout layer: rate=0.5
# 6. GlobalMaxPooling1D layer
# 7. Dense layer: 128 units, activation='relu'
# 8. Dropout layer: rate=0.5
# 9. Dense layer: 64 units, activation='relu'
# 10. Dropout layer: rate=0.3
# 11. Dense layer: num_classes units, activation='softmax'


model = Sequential([
    # 1. Embedding layer
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),

    # 2. Conv1D layer for local feature extraction
    Conv1D(filters=CNN_FILTERS, kernel_size=5, activation='relu'),

    # 3. MaxPooling1D layer to reduce dimensionality
    MaxPooling1D(pool_size=2),

    # 4. Bidirectional LSTM layer for sequence dependency
    Bidirectional(LSTM(LSTM_UNITS, return_sequences=True)),

    # 5. Dropout to prevent overfitting
    Dropout(0.5),

    # 6. GlobalMaxPooling1D to condense the sequence into a single vector
    GlobalMaxPooling1D(),

    # 7. Dense layer for deeper feature processing
    Dense(128, activation='relu'),

    # 8. Dropout
    Dropout(0.5),

    # 9. Dense layer
    Dense(64, activation='relu'),

    # 10. Dropout
    Dropout(0.3),

    # 11. Final Dense layer with softmax for multi-class classification
    Dense(num_classes, activation='softmax')
])











# Compile the model with:
# optimizer='adam'
# loss='categorical_crossentropy'
# metrics=['accuracy']
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)




# Print "Model Architecture:" and display model summary
print("Model Architecture:")
model.summary()



Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Step 11: Train the Model

**Task:** Set up callbacks and train the hybrid CNN-LSTM model.

In [17]:
# Create an EarlyStopping callback with:
# monitor='val_loss', patience=5, restore_best_weights=True, verbose=1

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)



# Create a ModelCheckpoint callback with:
# filepath='best_ticket_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1
checkpoint = ModelCheckpoint(
    filepath='best_ticket_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)




# Create a ReduceLROnPlateau callback with:
# monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)




# Print "Training the model..."
print("Training the model...")

# Train the model using fit() with:
# X_train, y_train as data
# epochs=30
# batch_size=32
# validation_split=0.2
# callbacks=[early_stopping, model_checkpoint, reduce_lr]
# verbose=1
# Store the result in a variable called history
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint, reduce_lr],
    verbose=1
)




# Print "\nTraining completed!"
print("\nTraining completed!")


Training the model...
Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.2557 - loss: 1.3833 
Epoch 1: val_accuracy improved from -inf to 0.48148, saving model to best_ticket_model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 343ms/step - accuracy: 0.2560 - loss: 1.3834 - val_accuracy: 0.4815 - val_loss: 1.3631 - learning_rate: 0.0010
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.3342 - loss: 1.3537
Epoch 2: val_accuracy did not improve from 0.48148
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.3378 - loss: 1.3539 - val_accuracy: 0.4815 - val_loss: 1.3336 - learning_rate: 0.0010
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.3882 - loss: 1.3238
Epoch 3: val_accuracy did not improve from 0.48148
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.3868 - loss: 1.3264 - val_accuracy: 0.4815 - val_loss: 1.2991 - learning_rate: 0.0010
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.3268 - loss: 1.3009
Epoch 4: val_accuracy did not 

## Step 12: Evaluate the Model

**Task:** Evaluate model performance on test data and display detailed metrics.

In [18]:
# Evaluate the model on X_test and y_test with verbose=0
# Store the results in test_loss and test_accuracy
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the test loss with 4 decimal places
print(f"Test Loss: {test_loss:.4f}")

# Print the test accuracy with 4 decimal places
print(f"Test Accuracy: {test_accuracy:.4f}")

# Make predictions on X_test using model.predict(), store in y_pred_proba
y_pred_proba = model.predict(X_test)

# Get the predicted class by finding argmax along axis=1, store in y_pred
y_pred = np.argmax(y_pred_proba, axis=1)

# Get the true class by finding argmax of y_test along axis=1, store in y_true
y_true = np.argmax(y_test, axis=1)

# Print "\nClassification Report:" header
print("\nClassification Report:")

# Print the classification report using y_true and y_pred with target_names from label_encoder.classes_
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


# Print "\nConfusion Matrix:" header
print("\nConfusion Matrix:")

# Calculate confusion matrix using y_true and y_pred, store in cm
cm = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print(cm)

# Create a DataFrame cm_df from cm with index and columns as label_encoder.classes_
cm_df = pd.DataFrame(
    cm,
    index=label_encoder.classes_,
    columns=label_encoder.classes_
)

# Print "\nConfusion Matrix with Labels:" header
print("\nConfusion Matrix with Labels:")

# Print the labeled confusion matrix DataFrame
print(cm_df)


Test Loss: 1.0715
Test Accuracy: 0.6061
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 637ms/step

Classification Report:
                   precision    recall  f1-score   support

          Billing       0.00      0.00      0.00         3
 Customer Service       0.43      0.60      0.50        10
            Sales       0.00      0.00      0.00         6
Technical Support       0.74      1.00      0.85        14

         accuracy                           0.61        33
        macro avg       0.29      0.40      0.34        33
     weighted avg       0.44      0.61      0.51        33


Confusion Matrix:
[[ 0  2  0  1]
 [ 0  6  0  4]
 [ 0  6  0  0]
 [ 0  0  0 14]]

Confusion Matrix with Labels:
                   Billing  Customer Service  Sales  Technical Support
Billing                  0                 2      0                  1
Customer Service         0                 6      0                  4
Sales                    0                 6      0                  0
Technical Support        0 

## Step 13: Save Model and Required Objects

**Task:** Save all artifacts needed for deployment in the Streamlit application.

In [19]:
# Save the trained model
model.save('ticket_classifier_model.keras')
print("Model saved as 'ticket_classifier_model.keras'")

# Save tokenizer
with open('ticket_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("Tokenizer saved as 'ticket_tokenizer.pkl'")

# Save label encoder
with open('ticket_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("Label encoder saved as 'ticket_label_encoder.pkl'")

# Save preprocessing parameters
preprocessing_params = {
    'max_sequence_length': MAX_SEQUENCE_LENGTH,
    'max_words': MAX_WORDS,
    'stop_words': list(stop_words)
}

with open('ticket_preprocessing_params.pkl', 'wb') as f:
    pickle.dump(preprocessing_params, f)
print("Preprocessing parameters saved as 'ticket_preprocessing_params.pkl'")

print("\nAll artifacts saved successfully!")

Model saved as 'ticket_classifier_model.keras'
Tokenizer saved as 'ticket_tokenizer.pkl'
Label encoder saved as 'ticket_label_encoder.pkl'
Preprocessing parameters saved as 'ticket_preprocessing_params.pkl'

All artifacts saved successfully!


## Step 14: Test Prediction Function

**Task:** Create a prediction function for ticket classification and test it with sample tickets.

In [20]:
def predict_department(text, model, tokenizer, label_encoder, max_len):
    """
    Predict department for a given support ticket
    """
    # Preprocess text
    cleaned = clean_text(text)
    processed = tokenize_and_lemmatize(cleaned)

    # Tokenize and pad
    sequence = tokenizer.texts_to_sequences([processed])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')

    # Predict
    prediction = model.predict(padded, verbose=0)
    predicted_class = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class]

    # Get department label
    department = label_encoder.classes_[predicted_class]

    return department, confidence, prediction[0]

# Test with sample tickets
test_tickets = [
    "My application keeps crashing when I try to export data. Need urgent technical help!",
    "I would like to know more about your enterprise pricing plans and volume discounts.",
    "I was charged twice on my credit card for the same subscription. Please refund the duplicate charge.",
    "Thank you for the excellent support! My issue was resolved quickly and professionally.",
    "Cannot login to my account. Password reset is not working. This is urgent!",
    "What are the contract terms for annual subscriptions? Need information before purchase."
]

print("Testing prediction function:\n")
for ticket in test_tickets:
    department, confidence, probs = predict_department(
        ticket, model, tokenizer, label_encoder, MAX_SEQUENCE_LENGTH
    )
    print(f"Ticket: {ticket}")
    print(f"Predicted Department: {department}")
    print(f"Confidence: {confidence:.4f}")
    print(f"All probabilities: {dict(zip(label_encoder.classes_, probs))}")
    print("-" * 100)


Testing prediction function:

Ticket: My application keeps crashing when I try to export data. Need urgent technical help!
Predicted Department: Technical Support
Confidence: 0.7068
All probabilities: {'Billing': np.float32(0.075473905), 'Customer Service': np.float32(0.13059103), 'Sales': np.float32(0.0871725), 'Technical Support': np.float32(0.70676255)}
----------------------------------------------------------------------------------------------------
Ticket: I would like to know more about your enterprise pricing plans and volume discounts.
Predicted Department: Customer Service
Confidence: 0.3164
All probabilities: {'Billing': np.float32(0.19953302), 'Customer Service': np.float32(0.316415), 'Sales': np.float32(0.24276756), 'Technical Support': np.float32(0.2412844)}
----------------------------------------------------------------------------------------------------
Ticket: I was charged twice on my credit card for the same subscription. Please refund the duplicate charge.
Predic